# Overview, Project Goals and Motivation

The motivation behind this project is to do stuff


## Outline
Part 1 - Data Extraction and Preprocessing
Part 2 - Exploratory Data Analysis
Part 3 - Final Analysis

# Related Work

Some related work

# Initial Questions


# Part 1 - Data extraction and Preprocessing
Data was extracted through the TMDB API. The code for this is located inside of the scraper.py file on the Github page.
At first we downloaded all movie posters possible including movies that were not in english and of any date, but then we decided to narrow down the variables and only accepted movie posters that were made for english speaking movies after the year 2000. We also limited the number of movies in each genre since we did not want a large imbalance in movies

In [1]:
######### IMPORTS ############
#data analysis and vis imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from mpl_toolkits.mplot3d import Axes3D

#dominant color imports
from colorthief import ColorThief

#machine learning imports
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.preprocessing import image
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier

#image imports
from PIL import Image 
import cv2

import sys
import os

### Part 1.1 - Dataset Visualization

# Part 2 - Exploratory Data Analysis

### Part 2.1 - Dominant Color Extraction

### Part 2.2 - Facial Recognition and Poster 

### Part 2.3 - Convolutional Neural Network

# Part 3 - Final Analysis